In [1]:
import pandas as pd
from pyvpsolver import *

# source used https://github.com/fdabrandao/vpsolver

# Preprocess

In [2]:
df_orders = pd.read_csv('demand_big.csv', delimiter='\t')
df_orders.columns = ['bin_length', 'quantity', 'subtotal']
orders_length = df_orders['bin_length'].values
orders_q = df_orders['quantity'].values


df_available = pd.read_csv('actual_big.csv', delimiter='\t')
available_length = df_available['wire_length'].values
available_q = df_available['quantity'].values

def BinPacking(w, q):
    s=[]
    for j in range(len(w)):
        for i in range(q[j]):
            s.append(w[j])
    return s

actual_length = BinPacking(available_length, available_q)
actual_length = [[i]for i in actual_length]
Ws = actual_length
Cs = [1]*len(actual_length)
Qs = [1]*len(actual_length)
ws = [[[x]] for x in orders_length]
b = orders_q.tolist()

instance = MVP(Ws, Cs, Qs, ws, b)
afg = AFG(instance)

VPSolver 3.1.2, Copyright (C) 2013-2016, Filipe Brandao
Build (method = -3)
  #dp: 148
  Step-3' Graph: 97 vertices and 2383 arcs (0.00s)
  Step-4' Graph: 91 vertices and 215 arcs (0.00s)
  #V4/#V3 = 0.94
  #A4/#A3 = 0.09
Ready! (0.00s)


# Solution

In [3]:
output, solution = VPSolver.script("vpsolver_scip.sh", instance)

Copyright (C) 2013-2016, Filipe Brandao

>>> vbp2afg...
VPSolver 3.1.2, Copyright (C) 2013-2016, Filipe Brandao
Build (method = -3)
  #dp: 148
  Step-3' Graph: 97 vertices and 2383 arcs (0.00s)
  Step-4' Graph: 91 vertices and 215 arcs (0.00s)
  #V4/#V3 = 0.94
  #A4/#A3 = 0.09
Ready! (0.00s)

>>> afg2mps...
VPSolver 3.1.2, Copyright (C) 2013-2016, Filipe Brandao
Generating the .MPS model...DONE!

>>> solving the MIP model using SCIP...
Note: different parameter settings may improve the performance substantially!
SCIP version 6.0.0 [precision: 8 byte] [memory: block] [mode: optimized] [LP solver: SoPlex 4.0.0] [GitHash: 77d3bc8]
Copyright (C) 2002-2018 Konrad-Zuse-Zentrum fuer Informationstechnik Berlin (ZIB)

External codes: 
  Readline EditLine w  GNU library for command line editing (gnu.org/s/readline)
  SoPlex 4.0.0         Linear Programming Solver developed at Zuse Institute Berlin (soplex.zib.de) [GitHash: 82cab95]
  CppAD 20180000.0     Algorithmic Differentiation of C++ algori

Bins of type 12: 1
1 x [i=3, i=5]
Bins of type 13: 1
1 x [i=3, i=5]
Bins of type 14: 1
1 x [i=1, i=6]
Bins of type 15: 1
1 x [i=1, i=6]
Bins of type 16: 1
1 x [i=1, i=3, i=4]
Bins of type 17: 1
1 x [i=4, i=4]
Bins of type 18: 1
1 x [i=4, i=4]
Bins of type 19: 1
1 x [i=2, i=6]
Bins of type 20: 1
1 x [i=3, i=6]
Bins of type 21: 1
1 x [i=3, i=6]
Bins of type 22: 1
1 x [i=3, i=6]
Bins of type 23: 1
1 x [i=3, i=6]
Bins of type 24: 1
1 x [i=3, i=6]
Bins of type 25: 1
1 x [i=3, i=6]
Bins of type 26: 1
1 x [i=3, i=6]
Bins of type 27: 1
1 x [i=3, i=6]
Bins of type 28: 1
1 x [i=4, i=5]
Bins of type 29: 1
1 x [i=4, i=5]
Bins of type 30: 1
1 x [i=4, i=5]
Bins of type 31: 1
1 x [i=4, i=5]
Bins of type 32: 1
1 x [i=4, i=5]
Bins of type 33: 1
1 x [i=4, i=5]
Bins of type 34: 1
1 x [i=4, i=5]
Bins of type 35: 1
1 x [i=1, i=1, i=6]
Bins of type 36: 1
1 x [i=1, i=1, i=2, i=2, i=2]
Bins of type 37: 1
1 x [i=1, i=1, i=2, i=2, i=2]
Bins of type 38: 1
1 x [i=1, i=1, i=2, i=2, i=2]
Bins of type 39: 1
1 x [i=1

In [4]:
df_1 = pd.DataFrame(solution[1])
df_1.columns = ['raw_solution']
bin_len = [44, 53, 60, 105, 120, 157]

In [5]:
def get_scrap(x):
    if x < min(bin_len):
        return x
    else:
        return 0

def get_bin_size(x):
    if x != None:
        x = x[1]
        result = []
        for i in x:
            result.append(bin_len[i[0]])
        return result
    else:
        return 0

def get_leftover(x):
    if x[1] == 0:
        return 0
    else:
        return x[0] - sum(x[1])
    
df_1['wire_len'] = actual_length
df_1['bins'] = df_1.apply(lambda x: get_bin_size(x[0]), axis = 1)
df_1['wire_len'] = df_1.apply(lambda x: (x[1][0]), axis = 1)
df_1 = df_1.drop(columns='raw_solution')
df_1['leftover'] = df_1.apply(lambda x: get_leftover(x) ,axis = 1)
df_1['scrap'] = df_1.apply(lambda x: get_scrap(x[2]), axis=1)

In [6]:
df_1

,wire_len,bins,leftover,scrap
0,94,0,0,0
1,105,0,0,0
2,107,0,0,0
3,118,0,0,0
4,124,0,0,0
5,159,0,0,0
6,159,0,0,0
7,160,0,0,0
8,165,0,0,0
9,167,"[60, 105]",2,2


In [7]:
bins_count = df_1['bins'].values.tolist()
bins_count = [i for i in bins_count if i != 0]

bin_times = sum(bins_count, [])
bin_dict = {}
for i in bin_times:
    if i not in bin_dict:
        bin_dict[i] = 1
    else:
        bin_dict[i] += 1

In [8]:
bin_dict

{60: 15, 105: 15, 53: 15, 120: 15, 44: 15, 157: 15}

In [9]:
df_1['scrap'].sum()

83

In [10]:
df_1.to_excel('Big_input_results_PySCIP.xlsx')